In [1]:
import geopandas
import json

# Define the initial CityJSON file
citymodel = "montreal.json"

# Define the roads dataset
roads_file = "roads.gpkg"

output = "montral_surfaces.json"

# Open files

Let's open the CityJSON file:

In [2]:
with open("montreal.json") as file:
    cm = json.load(file)

len(list(cm["CityObjects"]))

28018

Let's open the initial roads file:

In [3]:
roads = geopandas.read_file(roads_file)

len(roads)

4387

In [4]:
roads["is_intersection"] = roads.apply(lambda f: f["road_type"] == "Intersection", axis=1)
roads

,ID_VOI_VOI,road_type,geometry,is_intersection
0,200133764,Main road,"MULTIPOLYGON (((298480.212 5042188.495, 298480...",False
1,200401802,Main road,"MULTIPOLYGON (((298359.341 5042338.819, 298359...",False
2,100056164,Main road,"MULTIPOLYGON (((298195.252 5042469.794, 298195...",False
3,200408819,Main road,"MULTIPOLYGON (((298333.440 5042283.824, 298333...",False
4,100056284,Main road,"MULTIPOLYGON (((298144.546 5042698.956, 298144...",False
...,...,...,...,...
4382,210409690,Not treated,"MULTIPOLYGON (((298591.355 5043605.433, 298595...",False
4383,210409692,Not treated,"MULTIPOLYGON (((298594.327 5043601.656, 298604...",False
4384,210409694,Not treated,"MULTIPOLYGON (((298619.977 5043640.334, 298622...",False
4385,210409695,Not treated,"MULTIPOLYGON (((298622.546 5043644.396, 298633...",False


Dissolve the intersection to big polygons and assign a new id:

In [5]:
inter_dissolved = roads[roads["is_intersection"]].dissolve().explode()

inter_dissolved["inter_id"] = inter_dissolved.index.get_level_values(1)

inter_dissolved

ID_VOI_VOI     road_type  is_intersection  \
0 0     230303115  Intersection             True   
  1     230303115  Intersection             True   
  2     230303115  Intersection             True   
  3     230303115  Intersection             True   
  4     230303115  Intersection             True   
...           ...           ...              ...   
  359   230303115  Intersection             True   
  360   230303115  Intersection             True   
  361   230303115  Intersection             True   
  362   230303115  Intersection             True   
  363   230303115  Intersection             True   

                                                geometry  inter_id  
0 0    POLYGON ((299564.680 5042000.253, 299565.090 5...         0  
  1    POLYGON ((299359.932 5042000.000, 299354.167 5...         1  
  2    POLYGON ((298140.629 5042000.000, 298140.921 5...         2  
  3    POLYGON ((298667.133 5042002.686, 298668.709 5...         3  
  4    POLYGON ((299770.320 5042000.000, 299771.520 5...         4  
...                                                  ...       ...  
  359  POLYGON ((298801.689 5043003.681, 298801.642 5...       359  
  360  POLYGON ((299103.006 5042996.011, 299098.373 5...       360  
  361  POLYGON ((299792.406 5043006.720, 299791.222 5...       361  
  362  POLYGON ((298548.394 5043011.116, 298548.233 5...       362  
  363  POLYGON ((298069.838 5043012.610, 298073.863 5...       363  

[364 rows x 5 columns]

In [6]:
intersection_groups = geopandas.overlay(inter_dissolved, roads[roads["is_intersection"]], how="intersection")
intersection_groups

<ipython-input-6-660b6ff287e2>:1: UserWarning: `keep_geom_type=True` in overlay resulted in 31 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection_groups = geopandas.overlay(inter_dissolved, roads[roads["is_intersection"]], how="intersection")


,ID_VOI_VOI_1,road_type_1,is_intersection_1,inter_id,ID_VOI_VOI_2,road_type_2,is_intersection_2,geometry
0,230303115,Intersection,True,0,130077991,Intersection,True,"POLYGON ((299564.600 5042000.000, 299551.635 5..."
1,230303115,Intersection,True,0,230039361,Intersection,True,"POLYGON ((299564.680 5042000.253, 299565.090 5..."
2,230303115,Intersection,True,1,130077378,Intersection,True,"POLYGON ((299359.932 5042000.000, 299354.167 5..."
3,230303115,Intersection,True,1,130077382,Intersection,True,"POLYGON ((299354.167 5042000.000, 299347.893 5..."
4,230303115,Intersection,True,1,130077977,Intersection,True,"POLYGON ((299351.414 5042005.431, 299351.824 5..."
...,...,...,...,...,...,...,...,...
1280,230303115,Intersection,True,362,130077108,Intersection,True,"POLYGON ((298534.181 5043020.252, 298537.061 5..."
1281,230303115,Intersection,True,362,130077137,Intersection,True,"POLYGON ((298538.458 5043011.325, 298531.301 5..."
1282,230303115,Intersection,True,362,130077213,Intersection,True,"POLYGON ((298548.394 5043011.116, 298548.233 5..."
1283,230303115,Intersection,True,362,130077168,Intersection,True,"POLYGON ((298544.117 5043020.042, 298544.185 5..."


In [7]:
inter_ids = intersection_groups.dissolve(by="inter_id", aggfunc="first").reset_index()[["inter_id", "ID_VOI_VOI_2"]]
inter_ids

,inter_id,ID_VOI_VOI_2
0,0,130077991
1,1,130077378
2,2,130078172
3,3,130076549
4,4,230038592
...,...,...
359,359,130077202
360,360,130078312
361,361,130077987
362,362,130077108


In [8]:
roads["road_type"] = roads.apply(lambda f: "Main Intersection" if f["ID_VOI_VOI"] in list(inter_ids["ID_VOI_VOI_2"]) else f["road_type"], axis=1)
roads

,ID_VOI_VOI,road_type,geometry,is_intersection
0,200133764,Main road,"MULTIPOLYGON (((298480.212 5042188.495, 298480...",False
1,200401802,Main road,"MULTIPOLYGON (((298359.341 5042338.819, 298359...",False
2,100056164,Main road,"MULTIPOLYGON (((298195.252 5042469.794, 298195...",False
3,200408819,Main road,"MULTIPOLYGON (((298333.440 5042283.824, 298333...",False
4,100056284,Main road,"MULTIPOLYGON (((298144.546 5042698.956, 298144...",False
...,...,...,...,...
4382,210409690,Not treated,"MULTIPOLYGON (((298591.355 5043605.433, 298595...",False
4383,210409692,Not treated,"MULTIPOLYGON (((298594.327 5043601.656, 298604...",False
4384,210409694,Not treated,"MULTIPOLYGON (((298619.977 5043640.334, 298622...",False
4385,210409695,Not treated,"MULTIPOLYGON (((298622.546 5043644.396, 298633...",False


In [9]:
roads[roads["road_type"] == "Main Intersection"]

,ID_VOI_VOI,road_type,geometry,is_intersection
423,230303115,Main Intersection,"MULTIPOLYGON (((298505.531 5042555.627, 298501...",True
424,230303117,Main Intersection,"MULTIPOLYGON (((298877.851 5042314.206, 298873...",True
425,130077346,Main Intersection,"MULTIPOLYGON (((298296.971 5042168.063, 298297...",True
426,130077736,Main Intersection,"MULTIPOLYGON (((298195.320 5042469.750, 298198...",True
427,130077994,Main Intersection,"MULTIPOLYGON (((298998.063 5042824.491, 299006...",True
...,...,...,...,...
3988,230404827,Main Intersection,"MULTIPOLYGON (((298037.126 5043099.712, 298032...",True
3992,230180946,Main Intersection,"MULTIPOLYGON (((298005.253 5043063.123, 298006...",True
3993,230182040,Main Intersection,"MULTIPOLYGON (((298175.012 5043616.485, 298176...",True
3995,230180947,Main Intersection,"MULTIPOLYGON (((298072.802 5043143.236, 298075...",True


We divide the features to main roads and secondary roads. All secondary roads will be "assigned" to the main road with the biggest common boundary:

In [10]:
def get_parents(main_roads, secondary_roads):
    # Find common boundaries
    bgt_common_boundaries = geopandas.overlay(secondary_roads, main_roads, how='intersection', keep_geom_type=False)
    
    # Get the longest common boundary of every child
    bgt_common_boundaries['length'] = bgt_common_boundaries['geometry'].length
    max_rows = bgt_common_boundaries[['ID_VOI_VOI_1', 'road_type_1', 'ID_VOI_VOI_2', 'geometry', 'length']].groupby('ID_VOI_VOI_1')['length'].idxmax()
    temp_roads = bgt_common_boundaries.loc[max_rows]
    
    return temp_roads

def append_parent(f):
    """Appends the parent geometry with this one as a semantic surface"""
    child_id = f["ID_VOI_VOI_1"]
    parent_id = f["ID_VOI_VOI_2"]
    
    parent = cm["CityObjects"][str(parent_id)]
    parent_geom = parent["geometry"][0]
    
    # Create a surface for the current surfaces
    if not "semantics" in parent_geom:
        parent_geom["semantics"] = {"surfaces": [{"type": parent["attributes"]["road_type"]}], "values": [0 for v in parent_geom["boundaries"]]}
    
    new_i = len(parent_geom["semantics"]["surfaces"])
    
    # Append new surfaces from child
    child = cm["CityObjects"][str(child_id)]
    child_geom = child["geometry"][0]
    
    parent_geom["semantics"]["surfaces"].append({"type": child["attributes"]["road_type"], "id_voi_voi": str(child_id)})
    
    for b in child_geom["boundaries"]:
        parent_geom["boundaries"].append(b)
        parent_geom["semantics"]["values"].append(new_i)
    
    # Delete child object
    del cm["CityObjects"][str(child_id)]

main_roads = roads[roads["road_type"].isin(["Main road"])]
secondary_roads = roads[~roads["road_type"].isin(["Main road", "Main Intersection", "Intersection"])]
temp_roads = get_parents(main_roads, secondary_roads)
temp_roads.apply(append_parent, axis=1)

225     None
1881    None
1273    None
1869    None
1300    None
        ... 
1141    None
1101    None
1100    None
230     None
229     None
Length: 1745, dtype: object

In [11]:
main_roads = roads[roads["road_type"] == "Main Intersection"]
secondary_roads = roads[roads["road_type"] == "Intersection"]
temp_roads = get_parents(main_roads, secondary_roads)
temp_roads.apply(append_parent, axis=1)

323    None
544    None
545    None
558    None
112    None
       ... 
162    None
163    None
673    None
674    None
675    None
Length: 916, dtype: object

In [12]:
temp_roads

,ID_VOI_VOI_1,road_type_1,is_intersection_1,ID_VOI_VOI_2,road_type_2,is_intersection_2,geometry,length
323,130075951,Intersection,True,130075975,Main Intersection,True,"LINESTRING (299244.079 5043995.631, 299242.838...",2.231839
544,130075952,Intersection,True,130076037,Main Intersection,True,"LINESTRING (299440.674 5042290.374, 299439.098...",1.880000
545,130075953,Intersection,True,130076037,Main Intersection,True,"LINESTRING (299442.250 5042289.348, 299440.674...",1.880546
558,130075956,Intersection,True,130075955,Main Intersection,True,"LINESTRING (299376.732 5042205.295, 299374.408...",2.789042
112,130075958,Intersection,True,130076188,Main Intersection,True,"LINESTRING (298971.403 5042785.274, 298973.072...",2.958684
...,...,...,...,...,...,...,...,...
162,230405286,Intersection,True,130076509,Main Intersection,True,"POLYGON ((298945.390 5042428.974, 298947.948 5...",9.125053
163,230405288,Intersection,True,130076509,Main Intersection,True,POINT (298945.390 5042428.974),0.000000
673,230405289,Intersection,True,230405287,Main Intersection,True,POINT (299201.964 5042261.248),0.000000
674,230405290,Intersection,True,230405287,Main Intersection,True,"LINESTRING (299201.964 5042261.248, 299208.002...",8.012332


In [13]:
with open(output, "w") as out:
    json.dump(cm, out)

In [14]:
cm["CityObjects"]["130076037"]

{'attributes': {'id_voi_voi': '130076037', 'road_type': 'Intersection'},
 'geometry': [{'boundaries': [[[367917, 367919, 369402]],
    [[367917, 369403, 369404]],
    [[369402, 367556, 369403]],
    [[369403, 367556, 367557]],
    [[367917, 369402, 369403]],
    [[367918, 369404, 369403]],
    [[367918, 369403, 369422]],
    [[369422, 369403, 367558]],
    [[369422, 367558, 367560]],
    [[369403, 367557, 367558]]],
   'lod': 1,
   'type': 'MultiSurface',
   'semantics': {'surfaces': [{'type': 'Intersection'},
     {'type': 'Intersection', 'id_voi_voi': '130075952'},
     {'type': 'Intersection', 'id_voi_voi': '130075953'}],
    'values': [0, 0, 0, 0, 0, 1, 1, 2, 2, 2]}}],
 'type': 'Road'}